In [1]:
import pandas as pd

## Carregamento das bases de dados utilizadas

In [2]:
exportacao_geral = pd.read_excel('./dados/comex/Exportacao Mundo 2024 NCM.xlsx')
exportacao_geral['Código NCM'] = exportacao_geral['Código NCM'].astype(str)
exportacao_geral['Código NCM'] = exportacao_geral['Código NCM'].str.zfill(8)

exportacao_geral['sh6'] = exportacao_geral['Código NCM'].str[:6]

exportacao_geral = exportacao_geral.groupby('sh6')['2024 - Valor US$ FOB'].sum().reset_index()
exportacao_geral.rename(columns={'2024 - Valor US$ FOB': 'exportacao_mundo_2024_USD'}, inplace=True)

In [3]:
exportacao_geral.head()

,sh6,exportacao_mundo_2024_USD
0,010121,3134431
1,010129,5453342
2,010130,311
3,010190,1406
4,010221,2483913


In [4]:
exportacao_usa = pd.read_excel('./dados/comex/Exportacao EUA 2024 NCM.xlsx')

exportacao_usa['Código NCM'] = exportacao_usa['Código NCM'].astype(str)
exportacao_usa['Código NCM'] = exportacao_usa['Código NCM'].str.zfill(8)

exportacao_usa['sh6'] = exportacao_usa['Código NCM'].str[:6]
exportacao_usa = exportacao_usa.groupby('sh6')['2024 - Valor US$ FOB'].sum().reset_index()
exportacao_usa.rename(columns={'2024 - Valor US$ FOB': 'exportacao_eua_2024_USD'}, inplace=True)

In [5]:
exportacao_usa.head()

,sh6,exportacao_eua_2024_USD
0,010121,1713599
1,010129,3520510
2,010619,344
3,010620,24711
4,010639,1864


In [13]:
exportacao_geral.merge(exportacao_usa, on='sh6', how='left').fillna(0).to_excel('./dados/comex/Exportacao Mundo e EUA 2024 SH6.xlsx', index=False)

In [15]:
ncm_isic = pd.read_csv('./dados/comex/NCM_ISIC (1).csv',sep=';',dtype={'CO_ISIC_CLASSE': str, 'NO_ISIC_CLASSE': str}    )
ncm = pd.read_csv('./dados/comex/NCM (2).csv',sep=';',dtype={'CO_SH6': str, 'CO_ISIC_CLASSE': str})

ncnm_isic = ncm[['CO_SH6','CO_ISIC_CLASSE']].drop_duplicates().merge(ncm_isic[['CO_ISIC_CLASSE','NO_ISIC_CLASSE']], on='CO_ISIC_CLASSE', how='left')

In [9]:
ncnm_isic.head(2)

,CO_SH6,CO_ISIC_CLASSE,NO_ISIC_CLASSE
0,490110,5811,Publicação de livros
1,490191,5811,Publicação de livros


In [10]:
mapeamento_isic_cnae = pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx',dtype=str)
mapeamento_isic_cnae = mapeamento_isic_cnae.drop(columns=['Unnamed: 3']).rename(columns={'código ISIC': 'CO_ISIC_CLASSE', 'código CNAE': 'CNAE'})
mapeamento_isic_cnae = mapeamento_isic_cnae[['CO_ISIC_CLASSE', 'CNAE']]

In [11]:
mapeamento_isic_cnae.tail(4)

,CO_ISIC_CLASSE,CNAE
1185,U,U
1186,99,99
1187,990,99.0
1188,9900,99.00-8


In [27]:
ncm_isic = ncm_isic[['CO_ISIC_CLASSE']].drop_duplicates()

In [28]:
ncm_isic['CO_ISIC_CLASSE']

0      5310
1      5520
2      5811
3      5812
4      5813
       ... 
415    1393
416    1394
417    1399
418    1420
419    0990
Name: CO_ISIC_CLASSE, Length: 420, dtype: object

In [37]:
#Inserir a intensidade tecnológica dependendo do valor de início do CO_ISIC_CLASSE
prefixos_alta_complexidade = ("303","21","26")
prefixos_media_complexidade = ("252","29","325","28","20","27","30")
ncm_isic['Intensidade Tecnológica'] = None
ncm_isic['Intensidade Tecnológica'] = ncm_isic['CO_ISIC_CLASSE'].apply(
    lambda x: 'Alta intensidade' if x.startswith(prefixos_alta_complexidade) else
              'Média Alta' if x.startswith(prefixos_media_complexidade) and ~x.startswith("303") and ~x.startswith("301") else
              'Complexidade média ou abaixo'
)

/tmp/ipykernel_8707/4092528972.py:7: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  'Média Alta' if x.startswith(prefixos_media_complexidade) and ~x.startswith("303") and ~x.startswith("301") else


In [38]:
ncm_isic.query("`Intensidade Tecnológica`=='Média Alta'")['CO_ISIC_CLASSE'].str[0:3].drop_duplicates()

24     292
25     304
57     325
76     271
102    301
124    252
162    309
197    272
213    291
243    202
250    273
302    281
311    293
312    302
335    274
348    282
379    279
388    201
391    203
393    275
Name: CO_ISIC_CLASSE, dtype: object

In [39]:
ncm_isic.to_excel('./dados/mapeamento_isic_intensidade.xlsx',index=False)

In [103]:
#Remover códigos ISIC que possuem menos que 4 dígitos
mapeamento_isic_cnae = mapeamento_isic_cnae[mapeamento_isic_cnae['CO_ISIC_CLASSE'].str.len() >= 4]

In [104]:
mapeamento_isic_cnae['CO_ISIC_CLASSE'] = mapeamento_isic_cnae['CO_ISIC_CLASSE'].str.strip().str.zfill(4)

In [107]:
mapeamento_isic_cnae['CNAEd'] = mapeamento_isic_cnae['CNAE'].str.replace('.', '', regex=False).str[0:4]

In [111]:
ncnm_isic.query('CO_ISIC_CLASSE== "4530"')

,CO_SH6,CO_ISIC_CLASSE,NO_ISIC_CLASSE


In [118]:
ncnm_isic.merge(mapeamento_isic_cnae[['CO_ISIC_CLASSE','CNAEd']],on='CO_ISIC_CLASSE', how='left')[['CO_ISIC_CLASSE','CNAEd']].to_excel('./dados/mapeamento/ISIC_CNAE.xlsx', index=False)

## Tarifas harmonizadas

In [16]:
ncm_sh6 = ncm[['CO_SH6']].drop_duplicates()
ncm_sh6['CO_SH6'] = ncm_sh6['CO_SH6'].str.strip().str.zfill(6)


In [17]:
ncm_sh6.rename(columns={'CO_SH6': 'SH6'}, inplace=True)

In [1]:
import pandas as pd
harmonized = pd.read_excel('./dados/comex/Harmonized Tariff Schedule of the United States (HTSUS) not covered by the action.xlsx')

In [2]:
harmonized['SH6'] = harmonized['HTSUS'].str.replace('.', '', regex=False).str[0:6]

In [3]:
harmonized = harmonized.fillna('')

In [4]:
harmonized.head(2)

,HTSUS,Articles of Civil Aircraft only,Description,SH6
0,0801.21.00,,"Brazil nuts in shell, fresh or dried",080121
1,2008.30.35,,Orange pulp,200830


In [5]:
harmonized['Instrumento'] = 'Exclusão Ordem 30/07'
harmonized['Tarifa'] = 'Adicional de até 10%'

In [6]:
lista_aviacao = harmonized[['SH6','Articles of Civil Aircraft only']].query('`Articles of Civil Aircraft only`=="*"')
harmonized = harmonized.query('`Articles of Civil Aircraft only`!="*"')[['SH6','Instrumento','Tarifa']].drop_duplicates()


In [7]:
lista_aviacao = lista_aviacao.drop_duplicates()

In [8]:
harmonized

,SH6,Instrumento,Tarifa
0,080121,Exclusão Ordem 30/07,Adicional de até 10%
1,200830,Exclusão Ordem 30/07,Adicional de até 10%
2,200911,Exclusão Ordem 30/07,Adicional de até 10%
3,200912,Exclusão Ordem 30/07,Adicional de até 10%
5,252510,Exclusão Ordem 30/07,Adicional de até 10%
...,...,...,...
167,720260,Exclusão Ordem 30/07,Adicional de até 10%
168,720293,Exclusão Ordem 30/07,Adicional de até 10%
170,720310,Exclusão Ordem 30/07,Adicional de até 10%
171,720390,Exclusão Ordem 30/07,Adicional de até 10%


In [9]:
aco = pd.read_excel('./dados/comex/NCMs de aço e aluminio afetadas Seçao 232 EUA (2).xlsx',dtype=str).rename(columns={'SH_6' : 'SH6'})[['SH6']].drop_duplicates()

In [10]:

aco['Instrumento'] = 'Seção 232'
aco['Tarifa'] = "25%"







In [11]:
aco.head(2)

,SH6,Instrumento,Tarifa
0,720690,Seção 232,25%
1,720711,Seção 232,25%


In [116]:
#diam = pd.read_excel('./dados/comex/NCM_DIAM_Tarifas.xlsx',dtype=str).rename(columns={'SH_6' : 'SH6'})[['SH6']].drop_duplicates()


# Lê todas as planilhas do arquivo
arquivo_excel = './dados/comex/NCM_DIAM_Tarifas.xlsx'
planilhas = pd.read_excel(arquivo_excel, sheet_name=None,dtype=str)

automoveis = pd.DataFrame()
# `planilhas` é um dicionário: {nome_da_planilha: DataFrame}
for nome, df in planilhas.items():
    print(f'Planilha: {nome}')
    df.rename(columns={'Código NCM': 'NCM'}, inplace=True)
    df = df.rename(columns={'NCM': 'SH6'})
    df = df[['SH6']].drop_duplicates()
    df['Instrumento'] = 'Seção 232'
    df['Tarifa'] = '25%'
    df['SH6'] = df['SH6'].str.replace('.', '', regex=False)
    df['SH6'] = df['SH6'].str.zfill(8).str[:6]

    automoveis = pd.concat([automoveis, df], ignore_index=True)
    



Planilha: Automotivo
Planilha: Autopeças
Planilha: BK e BIT
Planilha: NAVAL


In [117]:
automoveis.head()

,SH6,Instrumento,Tarifa
0,842449,Seção 232,25%
1,842911,Seção 232,25%
2,842919,Seção 232,25%
3,842920,Seção 232,25%
4,842930,Seção 232,25%


In [131]:
ncm_sh6.shape

(6617, 1)

In [12]:
tarifas = pd.concat([harmonized[['SH6','Instrumento','Tarifa']],aco[['SH6','Instrumento','Tarifa']]])#,automoveis[['SH6','Instrumento','Tarifa']]])

In [13]:
tarifas = tarifas.drop_duplicates()

In [18]:
ncm_sh6 = ncm_sh6.merge(tarifas, on='SH6', how='left')

In [19]:
ncm_sh6 = ncm_sh6.merge(lista_aviacao, on='SH6', how='left')

In [20]:
ncm_sh6['Articles of Civil Aircraft only'] = ncm_sh6['Articles of Civil Aircraft only'].fillna('')
ncm_sh6['Instrumento'] = ncm_sh6['Instrumento'].fillna('Ordem 30/07')
ncm_sh6['Tarifa'] = ncm_sh6['Tarifa'].fillna('Tarifa de 50%')

In [21]:
ncm_sh6.to_excel('./dados/comex/NCM_SH6_Tarifas_semauto.xlsx', index=False)

In [153]:
ncm_sh6.query('SH6=="880100"')

,SH6,Instrumento,Tarifa,Articles of Civil Aircraft only
4620,880100,Ordem 30/07,Tarifa de 50%,*


In [151]:
ncm_sh6.query('`Articles of Civil Aircraft only`=="*"')[['Instrumento']].drop_duplicates()

,Instrumento
220,Seção 232
251,Ordem 30/07


In [152]:
ncm_sh6[['Instrumento']].drop_duplicates()

,Instrumento
0,Ordem 30/07
49,Seção 232
534,Exclusão Ordem 30/07


In [81]:
tarifas.drop_duplicates().query('SH6 == "903220"')

,SH6,Articles of Civil Aircraft only,Instrumento,Tarifa
645,903220,*,Exclusão Ordem 30/07,Adicional de até 10%
725,903220,,Seção 232,25%


In [76]:
tarifas.drop_duplicates().groupby('SH6')[['Instrumento']].count().sort_values(by='Instrumento', ascending=False).query('Instrumento >=2')

,Instrumento
SH6,
910910,2
910400,2
903290,2
903289,2
903220,2
...,...
730449,2
730441,2
730439,2


In [82]:
pd.concat([harmonized[['SH6','Articles of Civil Aircraft only','Instrumento','Tarifa']],aco[['SH6','Articles of Civil Aircraft only','Instrumento','Tarifa']],automoveis[['SH6','Articles of Civil Aircraft only','Instrumento','Tarifa']]]).drop_duplicates().to_excel('./dados/comex/dados_tarifas_sh6.xlsx', index=False)

In [67]:
harmonized.to_excel('./dados/comex/dados_tarifas_sh6.xlsx', index=False)

In [24]:
harmonized[['SH6','Articles of Civil Aircraft only']].to_excel('./dados/comex/HTSUS SH6.xlsx', index=False)

In [60]:
harmonized = harmonized[['SH6','Articles of Civil Aircraft only']].drop_duplicates()

In [63]:
exportacao_total = pd.read_excel('./dados/comex/Exportacao Mundo 2024 NCM.xlsx')

In [64]:
exportacao_total['SH6'] = exportacao_total['Código NCM'].astype(str).str.zfill(8).str[:6]

In [73]:
exportacao_usa = pd.read_excel('./dados/comex/Exportacao EUA 2024 NCM.xlsx')

exportacao_usa['SH6'] = exportacao_usa['Código NCM'].astype(str).str.zfill(8).str[:6]



In [76]:
exportacao_usa.head(2)

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,SH6
0,27090010,Óleos brutos de petróleo,5830983860,270900
1,72071200,Outros produtos semimanufaturados de ferro ou ...,2774470802,720712


In [68]:
ncm.head(2)

,CO_NCM,CO_UNID,CO_SH6,CO_PPE,CO_PPI,CO_FAT_AGREG,CO_CUCI_ITEM,CO_CGCE_N3,CO_SIIT,CO_ISIC_CLASSE,CO_EXP_SUBSET,NO_NCM_POR,NO_NCM_ESP,NO_NCM_ING
0,49011000,10,490110,3521.0,3521.0,3.0,89215,322,9000.0,5811,8099.0,"Livros, brochuras e impressos semelhantes, em ...","Libros, folletos, impresos similares, en hojas...","Books, brochures, similar printed papers, in s..."
1,49019100,10,490191,3521.0,3521.0,3.0,89216,322,9000.0,5811,8099.0,"Dicionários e enciclopédias, mesmo em fascículos","Diccionarios y enciclopedias, incluso en fascí...","Dictionaries and encyclopaedias, serial instal..."


In [77]:
exportacao_usa[['Código NCM','SH6']].drop_duplicates().merge(harmonized[['SH6','Articles of Civil Aircraft only']],on='SH6', how='inner')[['Código NCM']].drop_duplicates().shape

(626, 1)

In [61]:
exportacao_usa.head(2)

,sh6,exportacao_eua_2024_USD
0,010121,1713599
1,010129,3520510


In [26]:
expotacoes = harmonized[['SH6','Articles of Civil Aircraft only']].drop_duplicates().merge(exportacao_geral.merge(exportacao_usa, on='sh6', how='left').fillna(0), left_on='SH6',right_on='sh6', how='right')

In [27]:
expotacoes['Articles of Civil Aircraft only'] = expotacoes['Articles of Civil Aircraft only'].fillna('')

In [28]:
expotacoes['Excecao'] = ~expotacoes['SH6'].isnull()

In [29]:
expotacoes.drop(columns=['SH6']).to_excel('./dados/comex/HTSUS SH6 exportacao.xlsx', index=False)

In [ ]:
expotacoe

In [34]:
expotacoes.groupby('Excecao')[['exportacao_eua_2024_USD']].sum()

,exportacao_eua_2024_USD
Excecao,
False,2.238307e+10
True,1.798550e+10


In [30]:
expotacoes.drop(columns=['SH6'])

,Articles of Civil Aircraft only,sh6,exportacao_mundo_2024_USD,exportacao_eua_2024_USD,Excecao
0,,010121,3134431,1713599.0,False
1,,010129,5453342,3520510.0,False
2,,010130,311,0.0,False
3,,010190,1406,0.0,False
4,,010221,2483913,0.0,False
...,...,...,...,...,...
4890,,970400,195716,50.0,False
4891,,970510,1962,1962.0,False
4892,,970529,50,0.0,False
4893,,970610,861225,857301.0,False


In [31]:
harmonized[['SH6','Articles of Civil Aircraft only']].drop_duplicates().to_excel('./dados/comex/HTSUS SH6.xlsx', index=False)

In [32]:
harmonized[['SH6']].merge(ncm[['CO_SH6','CO_ISIC_CLASSE']], left_on='SH6', right_on='CO_SH6', how='left')

,SH6,CO_SH6,CO_ISIC_CLASSE
0,080121,080121,0125
1,200830,200830,1030
2,200911,200911,1030
3,200912,200912,1030
4,200912,200912,1030
...,...,...,...
2527,980200,NaN,NaN
2528,980200,NaN,NaN
2529,980200,NaN,NaN
2530,981800,NaN,NaN
